<a href="https://colab.research.google.com/github/acmilannesta/Bert-embedding/blob/master/GoogleQA_tfhub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Wed Dec 11 23:06:46 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.36       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    32W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
# bert uncased base
!wget https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1?tf-hub-format=compressed -O bbu
!mkdir bert_base_uncased
!tar -xzf bbu -C bert_base_uncased


# bert cased base
!wget https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/1?tf-hub-format=compressed -O bbc
!mkdir bert_base_cased
!tar -xzf bbc -C bert_base_cased

# albert base
!wget https://tfhub.dev/google/albert_base/1?tf-hub-format=compressed -O ab
!mkdir albert_base
!tar -xzf ab -C albert_base


--2019-12-11 23:06:55--  https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1?tf-hub-format=compressed
Resolving tfhub.dev (tfhub.dev)... 74.125.203.139, 74.125.203.113, 74.125.203.101, ...
Connecting to tfhub.dev (tfhub.dev)|74.125.203.139|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://storage.googleapis.com/tfhub-modules/tensorflow/bert_en_uncased_L-12_H-768_A-12/1.tar.gz [following]
--2019-12-11 23:07:00--  https://storage.googleapis.com/tfhub-modules/tensorflow/bert_en_uncased_L-12_H-768_A-12/1.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.125.128, 2404:6800:4008:c07::80
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.125.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 405800905 (387M) [application/x-tar]
Saving to: ‘bbu’

bbu                 100%[===================>] 387.00M  79.3MB/s    in 5.4s    

2019-12-11 23:07:06 (72.0 MB/s) - ‘bbu’ saved [405

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import sys
sys.path.insert(1, '/content/drive/My Drive/GoogleQA')
import numpy as np
import pandas as pd
import os, gc
import codecs
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras.backend import set_session
# from keras.callbacks import Callback
# from keras.models import Model, load_model
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold, GroupKFold, KFold
from scipy.stats import spearmanr
from tqdm import tqdm
import tensorflow_hub as hub
# from albert_tokenization import FullTokenizer
from tokenizer import Tokenizer
from warmup_v2 import AdamWarmup, calc_train_steps
tf.get_logger().setLevel('WARNING') 
# from keras_bert import load_trained_model_from_checkpoint, Tokenizer, AdamWarmup, calc_train_steps, get_custom_objects
# from hyperopt import fmin, hp, tpe, STATUS_OK, Trials


In [0]:
data_path = 'drive/My Drive/GoogleQA/Data/'
train = pd.read_csv(data_path+'train.csv')
test = pd.read_csv(data_path+'test.csv')
sub = pd.read_csv(data_path+'sample_submission.csv')

target_col = train.columns.tolist()[11:42]
train['question'] = train['question_title'] + ' ' + train['question_body']
test['question'] = test['question_title'] + ' ' + test['question_body']

def aux_build(df):
    aux = pd.get_dummies(df['category'], drop_first=True)
    aux['stackexchange'] = df.host.str.contains('stackexchange').astype('int32')
    aux['stackoverflow'] = df.host.str.contains('stackoverflow').astype('int32')
    return aux
train_aux, test_aux = aux_build(train), aux_build(test)
# aux1 = pd.get_dummies(df_aux['category'], drop_first=True)
# aux2 = pd.get_dummies(df_aux['host'].apply(lambda x: x.split('.')[0]), drop_first=True)
# aux = pd.concat([aux1, aux2], 1)
# train_aux = pd.concat([train, aux.iloc[:len(train), ]], 1)
# test_aux =  pd.concat([test, aux.iloc[len(train):, ]], 1)

In [0]:
MAXLEN = 512 #@param {type:"slider", min:128, max:512, step:32}
BATCH_SIZE = 4 #@param {type:'slider', min:1, max:32, step:1}
NUM_EPOCHS = 3
NUM_CLASSES = 30
LR = 5e-5
MIN_LR = 0
model_path = 'bert_base_uncased' #@param ['bert_base_uncased', 'bert_base_cased', 'albert_base']
save_model = 'bbu'
# OUTPUT_TRAIN = 'train_bert_ipredcv1415_oof.csv'
# OUTPUT_TEST = 'test_bert_large.npy'
# model_path = 'uncased_L-12_H-768_A-12' #@param ['uncased_L-12_H-768_A-12', 'wwm_uncased_L-24_H-1024_A-16', 'uncased_L-24_H-1024_A-16']
# target_q_col = train.columns.tolist()[11:32]
# target_a_col = train.columns.tolist()[32:42]


## Bert tokenizer

In [0]:
token_dict = {}
with codecs.open(os.path.join(model_path, 'assets/vocab.txt'), 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)
tokenizer = Tokenizer(token_dict)

def convert_data(data_df, branch='training'):
    data_df = data_df.reset_index(drop=True)
    global tokenizer
    global MAXLEN
    global target_col
    ids, segments = [], []
    # q_title, q_body, answer = [], [], []
    for i in tqdm(range(len(data_df))):
        a, b = tokenizer.encode(data_df.loc[i, 'question'], data_df.loc[i, 'answer'])
        ids.append(a)
        segments.append(b)                 
        # q_title_ids, _ = tokenizer.encode(data_df.loc[i, 'question'])
        # q_title.append(q_title_ids)
        # q_body_ids, _ = tokenizer.encode(data_df.loc[i, 'answer'])
        # q_body.append(q_body_ids)
    if branch == 'training':
        targets = data_df[target_col]
        # return [q_title, q_body], np.array(targets)
        return [ids, segments], np.array(targets)
    else:
        return [ids, segments]
        # return [q_title, q_body]

## Albert tokenizer

In [0]:
tokenizer = FullTokenizer(
    vocab_file=model_path+'/assets/30k-clean.vocab', 
    do_lower_case=True, 
    spm_model_file=model_path + '/assets/30k-clean.model')

def convert_data(data_df, branch='training'):
    data_df = data_df.reset_index(drop=True)
    global tokenizer
    global MAXLEN

    ids, segments = [], []
    for i in tqdm(range(len(data_df))):
        id = tokenizer.tokenize(data_df.loc[i, 'question']) + tokenizer.tokenize(data_df.loc[i, 'answer'])
        id = tokenizer.convert_tokens_to_ids(id)
        segment = [0] * len(data_df.loc[i, 'question']) + [1] * len(data_df.loc[i, 'answer'])
        ids.append(id)
        segments.append(segment)
    if branch == 'training':
        targets = data_df[target_col]
        # return [q_title, q_body], np.array(targets)
        return [ids, segments], np.array(targets)
    else:
        return [ids, segments]
        # return [q_title, q_body]

In [0]:
"""## Data Generator"""
def seq_padding(X, padding=0):
    L = [len(x) for x in X]
    ML = min(max(L), MAXLEN)
    return np.array([np.concatenate([x, [padding] * (ML - len(x))]) if len(x) < ML else x[:ML] for x in X])

def get_masks(X, padding=0):
    L = [len(x) for x in X]
    ML = min(max(L), MAXLEN)
    return np.array([np.concatenate([[1]*len(x), [padding] * (ML - len(x))]) if len(x) < ML else [1]*ML for x in X])


class data_generator:
    def __init__(self, data, batch_size=BATCH_SIZE, branch='train'):
        self.data = data
        self.batch_size = batch_size
        self.branch = branch
        # self.q_a = q_a
        self.steps = len(self.data) // self.batch_size
        if len(self.data) % self.batch_size != 0:
            self.steps += 1

    def __len__(self):
        return self.steps

    def __iter__(self):
        while True:
            if self.branch == 'train':
                np.random.shuffle(self.data)
            for i in range(self.steps):
                d = self.data[i * self.batch_size: (i + 1) * self.batch_size]
                X1 = seq_padding([x[0] for x in d])
                X2 = get_masks([x[0] for x in d])    
                X3 = seq_padding([x[1] for x in d])
                # X3 = np.zeros_like(X1)

                # X4 = seq_padding([x[1] for x in d])
                # X5 = get_masks([x[1] for x in d])    
                # X6 = np.zeros_like(X4)

                # X7 = seq_padding([x[2] for x in d])
                # X8 = get_masks([x[2] for x in d])    
                # X9 = np.zeros_like(X7)
                aux = np.array([x[2] for x in d])
                if self.branch == 'test':
                    # aux = np.array([x[3] for x in d])
                    yield [X1, X2, X3, aux]
                    # yield [X1, X2, X3, X4, X5, X6]
                else:
                    Y = np.array([x[3] for x in d])
                    # aux = np.array([x[4] for x in d])
                    yield [X1, X2, X3, aux], Y
                    # yield [X1, X2, X3, X4, X5, X6], Y


In [0]:
def model_build(len_train, model='albert'):
    global NUM_CLASSES
    global BATCH_SIZE
    global NUM_EPOCHS
    global MIN_LR
    global LR
    global MAXLEN
    global model_path
    global train_aux
    # bert_model = load_trained_model_from_checkpoint(
    #     os.path.join(model_path, 'bert_config.json'),
    #     os.path.join(model_path, 'bert_model.ckpt'),
    #     seq_len = MAXLEN,
    #     trainable=True,
    # )
    

    q_in = keras.layers.Input(shape=(None,), dtype=tf.int32, name="q_input_word_ids")
    q2_in = keras.layers.Input(shape=(None,), dtype=tf.int32, name="q_input_masks")
    q3_in = keras.layers.Input(shape=(None,), dtype=tf.int32, name="q_segment_ids")
    aux_in = keras.layers.Input(shape=(train_aux.shape[1],), name="aux")
    # qb_in = keras.layers.Input(shape=(None,), dtype=tf.int32, name="qb_input_word_ids")
    # qb2_in = keras.layers.Input(shape=(None,), dtype=tf.int32, name="qb_input_masks")
    # qb3_in = keras.layers.Input(shape=(None,), dtype=tf.int32, name="qb_segment_ids")

    # a_in = keras.layers.Input(shape=(None,), dtype=tf.int32, name="a_input_word_ids")
    # a2_in = keras.layers.Input(shape=(None,), dtype=tf.int32, name="a_input_masks")
    # a3_in = keras.layers.Input(shape=(None,), dtype=tf.int32, name="a_segment_ids")

    # _, qb_inputs  = bert_layer([qb_in, qb2_in, qb3_in])
    # qb_outputs = keras.layers.Lambda(lambda x: x[:, 0])(qb_inputs)

    # _, a_inputs  = bert_layer([a_in, a2_in, a3_in])
    # a_outputs = keras.layers.Lambda(lambda x: x[:, 0])(a_inputs)
    if model=='bert':
        bert_layer = hub.KerasLayer(model_path, trainable=True)
        _, q_inputs  = bert_layer([q_in, q2_in, q3_in])
        # q_outputs = keras.layers.Lambda(lambda x: x[:, 0])(q_inputs)
        q_outputs = keras.layers.GlobalAveragePooling1D()(q_inputs)
        dense = keras.layers.Dropout(0.2)(q_outputs)
        del bert_layer
        gc.collect()

    if model=='albert':
        albert_module = hub.Module(model_path, trainable=True)
        albert_inputs = dict(input_ids=q_in, input_mask=q2_in, segment_ids=q3_in)
        albert_outputs = albert_module(albert_inputs, signature="tokens", as_dict=True)["sequence_output"]
        sess = tf.Session()
        graph = tf.get_default_graph()
        dense = keras.layers.Lambda(lambda x: x[:, 0])(albert_outputs)
        del albert_module
        gc.collect()
    
    dense = keras.layers.concatenate([dense, aux_in])
    outputs = keras.layers.Dense(NUM_CLASSES, activation='sigmoid')(dense)
    model = keras.Model([q_in, q2_in, q3_in, aux_in], outputs)

    decay_steps, warmup_steps = calc_train_steps(
        len_train,
        batch_size=BATCH_SIZE,
        epochs=NUM_EPOCHS,
    )

    model.compile(
        loss='binary_crossentropy',
    #     optimizer=keras.optimizers.Adam(LR),
        optimizer=AdamWarmup(
            decay_steps=decay_steps,
            warmup_steps=warmup_steps,
            lr=LR,
            min_lr=MIN_LR,
            ))
    # )

    return model


## train model

In [8]:
test_x = convert_data(test, branch='testing')
pred = np.zeros((len(test), NUM_CLASSES))
kf = KFold(n_splits=5, shuffle=True, random_state=0)
idx = [x for x in kf.split(train)]

def compute_spearmanr(trues, preds):
    rhos = []
    for col_trues, col_pred in zip(trues.T, preds.T):
        rhos.append(spearmanr(col_trues, col_pred + np.random.normal(0, 1e-7, col_pred.shape[0])).correlation)
    return np.nanmean(rhos)

class IntervalEval(keras.callbacks.Callback):
    def __init__(self, val_data, test_data, label):
        global NUM_EPOCHS
        global save_model
        super(keras.callbacks.Callback, self).__init__()
        self.val_data = val_data
        self.test_data = test_data
        self.label = label
        self.total_epochs = NUM_EPOCHS
        self.test_pred = []
        self.score = []
        self.save_model = save_model
    def on_epoch_end(self, epoch, logs={}):
        # if epoch == 3:
        val_pred = self.model.predict_generator(self.val_data.__iter__(), len(self.val_data), verbose=1)
        score = compute_spearmanr(self.label, val_pred)
        self.score.append(score)
        print('Spearman - {:.5f}'.format(score))
        if epoch>=self.total_epochs-2:
            print('--Save Model--')
            self.model.save('drive/My Drive/GoogleQA/Models/{:}-{:}-{:}.h5'.format(self.save_model, i, epoch+1))
        #     print("Testing Pred")
        #     test_pred = self.model.predict_generator(self.test_data.__iter__(), len(self.test_data), verbose=1)
        #     self.test_pred.append(test_pred)
        # if epoch==self.total_epochs-1:
        #     if self.score[-2]<=self.score[-1]:
        #         self.test_pred = self.test_pred[0] * 0.3 + self.test_pred[1] * 0.7
        #     else:
        #         self.test_pred = self.test_pred[0] * 0.7 + self.test_pred[1] * 0.3

100%|██████████| 476/476 [00:01<00:00, 313.27it/s]


In [0]:
for i, (tr_idx, val_idx) in enumerate(idx[1:], 2):
    print('\nFold - {:}\n'.format(i))
    tr, val = train.loc[tr_idx], train.loc[val_idx]
    tr_aux, val_aux = train_aux.loc[tr_idx], train_aux.loc[val_idx]
    tr_x, tr_y = convert_data(tr)
    val_x, val_y = convert_data(val)
    
    model = model_build(len(tr), model='bert')

    train_D = data_generator(list(zip(tr_x[0], tr_x[1], tr_aux.values, tr_y)))
    valid_D = data_generator(list(zip(val_x[0], val_x[1], val_aux.values, val_y)), branch='valid')
    test_D = data_generator(list(zip(test_x[0], test_x[1], test_aux.values)), branch='test')
    ieval = IntervalEval(val_data=valid_D, label=val_y, test_data=test_D)
    model.fit_generator(
        train_D.__iter__(),
        steps_per_epoch=len(train_D),
        epochs=NUM_EPOCHS,
        callbacks = [ieval]
    )
    # pred+ = model.predict_generator(test_D.__iter__(), len(test_D), verbose=1) / kf.get_n_splits()
    # train_aug.loc[val_idx, 'oof_pred'] = np.argmax(oof_pred, 1)
    # print('oof - {:} f1_score - {:.4f}'.format(i, spearmanr(val_y, np.argmax(oof_pred, 1), average='weighted')))

    # pred+= ieval.test_pred / kf.get_n_splits()
    # np.save(OUTPUT_TEST, pred)
    # s3.upload_file(Filename=OUTPUT_TEST, Bucket='acmilannesta', Key='large/'+OUTPUT_TEST)

    # np.save('drive/My Drive/GoogleQA/pred', pred)
    # model.save('drive/My Drive/GoogleQA/Models/bertbase-{:}.h5'.format(i))

    del model, train_D, valid_D, test_D, ieval, tr, val, tr_x, tr_y, val_x, val_y
    gc.collect()

  1%|          | 28/4863 [00:00<00:17, 276.46it/s]


Fold - 2



100%|██████████| 1216/1216 [00:03<00:00, 333.89it/s]


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/3
304/304 [==============================] - 38s 125ms/step
Spearman - 0.36789
1216/1216 [==============================] - 505s 415ms/step - loss: 0.4044
Epoch 2/3
304/304 [==============================] - 28s 93ms/step
Spearman - 0.38569
--Save Model--
1216/1216 [==============================] - 435s 358ms/step - loss: 0.3631
Epoch 3/3
304/304 [==============================] - 28s 93ms/step
Spearman - 0.39556
--Save Model--
1216/1216 [==============================] - 429s 353ms/step - loss: 0.3387


  1%|          | 26/4863 [00:00<00:19, 251.79it/s]


Fold - 3



100%|██████████| 1216/1216 [00:10<00:00, 112.18it/s]


Epoch 1/3
 167/1216 [===>..........................] - ETA: 13:47 - loss: 0.4911

In [0]:
sub[target_col] = pred
sub.to_csv('drive/My Drive/GoogleQA/Submissions/sub_1.csv', index=False)